In [25]:
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np
from quantfreedom.backtester.nb.temp import simulate_df_only
from quantfreedom.backtester.base.base import rsi_below_entries
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.utils.caching import clear_cache

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv('30min.csv', index_col='time')
entries = rsi_below_entries(
    timeperiods=[20, 30],
    below_ranges=[40],
    prices=prices.close
)


In [18]:
np.array([entries.values[:, 0]]*3)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [26]:
entries.shape[1]

rsi_below               40       
rsi_timeperiod          20     30
time                             
2020-10-01 00:00:00  False  False
2020-10-01 00:30:00  False  False
2020-10-01 01:00:00  False  False
2020-10-01 01:30:00  False  False
2020-10-01 02:00:00  False  False
...                    ...    ...
2021-02-22 21:30:00  False  False
2021-02-22 22:00:00  False  False
2021-02-22 22:30:00  False  False
2021-02-22 23:00:00  False  False
2021-02-22 23:30:00  False  False

[6960 rows x 2 columns]

In [2]:
strat_array = simulate_df_only(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(1,7,1),
    risk_rewards=np.arange(1,11.1,.25),
    size_pct=1.,
    gains_pct_filter=1000,
    # sl_pcts=np.arange(1,6,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(1,11.1,.25),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,11,.25),
    tsl_when_pct_from_avg_entry=np.arange(1,11,.5),
)
pd.DataFrame(strat_array).sort_values(by='gains_pct', ascending=False)


In [ ]:
clear_cache()